# Supply Chain Maze

In [6]:
import cv2
import numpy as np
# pip install Pillow
    # open Anaconda Prompt and paste above line (without '#') to install package
from PIL import Image

In [8]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [19]:
# take from SCM Calibration
points = [[325, 45], [172, 146], [478, 153], [191, 228], [357, 232], [481, 237], [211, 332], [451, 328], [336, 417]]

color = [255, 0, 0]                 # color in BGR colorspace
line_color = [168, 45, 81]            # color used to draw path
line_size = 3                         # line thickness
line_color_check = [0, 255, 255]      # line color used to draw the correct solution
detection_size = 500                  # size of detection required to mark color as an object
capture = cv2.VideoCapture(0)         # picks camera to use (usually 0 or 1)

if (line_color == [0, 0, 0]): # breaks if you use black ([0, 0, 0]), so we adjust it
    line_color = [1, 1, 1]
if (line_color_check == [0, 0, 0]): # breaks if you use black ([0, 0, 0]), so we adjust it
    line_color_check = [1, 1, 1]

first = False
count = 0
prev_node = -1
current_node = -1
path_value = 0
                     # node   s   1   2   3   4   5   6   7   e
adjacency_matrix = np.array([[0,  4,  8,  0,  0,  0,  0,  0,  0 ],   # start
                             [4,  0,  11, 4,  0,  0,  0,  0,  0 ],   # 1
                             [8,  11, 0,  0,  2,  7,  0,  0,  0 ],   # 2
                             [0,  4,  0,  0,  1,  0,  7,  7,  0 ],   # 3
                             [0,  0,  2,  1,  0,  4,  0,  0,  0 ],   # 4
                             [0,  0,  7,  0,  4,  0,  0,  7,  0 ],   # 5
                             [0,  0,  0,  7,  0,  0,  0,  11, 8 ],   # 6
                             [0,  0,  0,  7,  0,  7,  11, 0,  11],   # 7
                             [0,  0,  0,  0,  0,  0,  8,  11, 0 ]])  # end
                # a '1' indicates that two nodes (corresponing to the row and column) are adjacent
                # the matrix is symmetric, so it doesn't matter if you take [3][4] or [4][3] ; they tell you the same thing
while True:
    ret, frame = capture.read()
    frame = cv2.flip(frame, 1)
    
    if (first == False): # runs only once
        # get height and width of video
        shape = frame.shape
        height = shape[0]
        width = shape[1]
        camera_quality_mod = width / 640  # scales detection with video quality (because it depends on number of pixels)
        line_thickness = int(line_size * camera_quality_mod)
        # create path image to draw on (based on shape of webcam feed)
        path = np.array(Image.new("RGB", (width, height), (0,0,0)))
        # draw dots to mark each node at each node
        for i in points:
            cv2.circle(path, i, 1, (0, 255, 255), int(5*camera_quality_mod))
        # set first to True so this block doesn't run again
        first = True

    frame_blur = cv2.GaussianBlur(frame, (11, 11), 9) # blurring the image may help get the desired result, but it can be removed
    
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV
    lowerLimit, upperLimit = get_limits(color) # range of hues that we want the software to detect
    
    mask = cv2.inRange(frame_hsv, lowerLimit, upperLimit) # detects objects in color range
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    bboxes = list()
    for cnt in contours:
        if cv2.contourArea(cnt) > camera_quality_mod*detection_size:  # only continues if size of the object is large enough (removes noise)
            x1, y1, w, h = cv2.boundingRect(cnt)  # finds a bounding box for each object
            c = list([int(x1 + w/2), int(y1 + h/2)])  # centerpoint of bbox
            # check other bboxes to see if we want to combine them into 1 box
            newBox = list([c[0], c[1], x1, y1, w, h, cv2.contourArea(cnt)])
            for i in bboxes[:]:
                cxi, cyi, x1i, y1i, wi, hi, si = i  # bbox we check the newBox against
                if np.sqrt((c[0] - cxi)**2 + (c[1] - cyi)**2) < np.sqrt(w**2 + h**2)/3 + np.sqrt(wi**2 + hi**2)/3 + camera_quality_mod*25: # if centerpoints are close enough (scales with box size)
                    bboxes.remove(i)
                    # reassign bbox boundaries so the new box contains both nearby boxes
                    newBox[2], newBox[3] = min(x1, x1i), min(y1, y1i)  # reassigns x1 and y1 values
                    newBox[4], newBox[5] = max(x1+w, x1i+wi) - newBox[2], max(y1+h, y1i+hi) - newBox[3]  # reassgins w and h values
                    newBox[0], newBox[1] = int(newBox[2] + newBox[4]/2), int(newBox[3] + newBox[5]/2)  # reassigns centerpoint values
            bboxes.append(newBox)

    # overlay the path over the webcam feed
    mask2 = cv2.cvtColor(path, cv2.COLOR_BGR2GRAY)
    mask2 = cv2.threshold(mask2, 0, 255, cv2.THRESH_BINARY)[1]
    mask2inv = cv2.bitwise_not(mask2)
    pathfg = cv2.bitwise_and(path, path, mask=mask2) # extract foreground
    framebg = cv2.bitwise_and(frame, frame, mask=mask2inv) # extract background
    frame = cv2.add(framebg, pathfg) # combine foreground and background
    
    # find the largest object
    maxVal = 0
    point = 0
    for i in bboxes:  
        cx, cy, x1, y1, w, h, s = i
        if s > maxVal:
            maxVal = s
            point = i

    # draw circle around current node
    if current_node != -1:
        cv2.circle(frame, points[current_node], int(20*camera_quality_mod), (0, 0, 255), int(3*camera_quality_mod))
        
    # draw bounding box
    if point != 0:
        cx, cy, x1, y1, w, h, s = point
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

        # check if we are close enough to a node
        for index, value in enumerate(points):
            if current_node == -1 and index == 0 and (point[0]-value[0])**2+(point[1]-value[1])**2 < camera_quality_mod*200:
                current_node = 0
            if adjacency_matrix[current_node][index] != 0 and index != prev_node and (point[0]-value[0])**2+(point[1]-value[1])**2 < camera_quality_mod*50:
                cv2.line(path, (points[current_node][0], points[current_node][1]), (points[index][0], points[index][1]), (0, 255, 255), line_thickness)
                prev_node = current_node
                current_node = index
                path_value += adjacency_matrix[prev_node][current_node]

    
    cv2.imshow('blur', frame_blur)
    cv2.imshow('mask', mask)
    cv2.imshow('webcam', frame)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:               # 27 is ASCE for [ESC]
        break                   # ends loop when [ESC] is pressed

capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when spacebar is pressed

print(path_value)

23
